# Proximal Causal Inference Synthetic Simulations \#2

In this set of experiments, we attempt to simulate text-based proximal causal inference when two pieces of text data are independent and representative of the underlying distribution of text data. Below are the steps of the experiment:

1. Make a training dataset DTrain. For each row of data in DTrain, generate two realizations of X1, X2, X3, and X4. 
2. Generate X = [mean(X11, X12), mean(X21, X22), mean(X31, X32), mean(X41, X42)]. Train a single linear logistic regression on this data to predict U. This simulates training a zero-shot classifier on some broad background/training data.
3. Make a dataset that simulates inference time DInference. For each row of data, generate two realizations of X1, X2, X3, and X4. These two realizations of X1 through X4 are independent by definition.
4. Apply the previously trained 'zero-shot model' to obtain Z and W from the two realizations of X1, X2, X3, and X4.
5. Proceed with proximal pipeline as usual.

In [54]:
import pandas as pd
import numpy as np
from scipy.special import expit
import statsmodels.api as sm

np.random.seed(0)
size = 20000

Code implementing future helper functions.

In [55]:
def binary_predictions(a):
    # takes array a of probabilities, and returns a prediction for 0s and 1s
    output = []

    for ele in a:
        if ele < 0.5:
            output.append(0)
        else:
            output.append(1)

    # output = np.random.binomial(1, a, len(a))

    return output

def proximal_find_ace(A, Y, W, Z, covariates, data):
    # fit a model W~A+Z
    formula = W+"~"+A+"+"+Z
    if len(covariates) > 0:
        formula += '+' + '+'.join(covariates)
    model1 = sm.GLM.from_formula(formula=formula, data=data, family=sm.families.Binomial()).fit()

    # make predictions for What
    What = model1.predict(data)
    data["What"] = What

    # fit a model Y~A+What
    formula = Y+"~"+A+"+What"
    if len(covariates) > 0:
        formula += '+' + '+'.join(covariates)
    model2 = sm.GLM.from_formula(formula=formula, data=data, family=sm.families.Gaussian()).fit()

    # the ACE is the coefficient for A in model2
    return model2.params[A]

def compute_confidence_intervals(A, Y, W, Z, covariates, data, num_bootstraps=200, alpha=0.05):
    """
    Compute confidence intervals for backdoor adjustment via bootstrap
    
    Returns tuple (q_low, q_up) for the lower and upper quantiles of the confidence interval.
    """
    
    Ql = alpha/2
    Qu = 1 - alpha/2
    # two lists for the two indexes of output
    estimates = []
    
    for i in range(num_bootstraps):
        
        # resample the data with replacement
        data_sampled = data.sample(len(data), replace=True)
        data_sampled.reset_index(drop=True, inplace=True)
        
        # add estimate from resampled data
        output = proximal_find_ace(A, Y, W, Z, covariates, data_sampled)
        estimates.append(output)

    # calculate the quantiles
    quantiles = np.quantile(estimates, q=[Ql, Qu])
    q_low = quantiles[0]
    q_up = quantiles[1]
    
    return (q_low, q_up)

def odds_ratio(X, Y, Z, data):
    # calculate the odds ratio assuming linearity

    formula = X + '~1+' + Y
    if len(Z) > 0:
        formula += '+' + '+'.join(Z)

    model = sm.GLM.from_formula(formula=formula, data=data, family=sm.families.Binomial()).fit()

    return np.exp(model.params[Y])

Define the data generating process.

In [56]:
def generate_data():
    U = np.random.binomial(1, 0.48, size)

    X11 = np.random.normal(0, 1, size) + 2*U
    X12 = np.random.normal(0, 1, size) + 2*U

    # make sure that X2 is some non-linear function
    X21 = np.random.normal(0, 1, size) + np.exp(X11) + U
    X22 = np.random.normal(0, 1, size) + np.exp(X12) + U

    X31 = np.random.normal(0, 1, size) + 1.3*U
    X32 = np.random.normal(0, 1, size) + 1.3*U

    # make sure that X4 is some non-linear function
    X41 = np.random.normal(0, 1, size) + X31**2 + 0.5*X31**3 + U
    X42 = np.random.normal(0, 1, size) + X32**2 + 0.5*X32**3 + U

    A = np.random.binomial(1, expit(0.8*U), size)

    Y = np.random.normal(0, 1, size) + 1.3*A + 1.4*U

    data = pd.DataFrame({"U": U, "X11": X11, "X21": X21, "X31": X31, "X41": X41, "X12": X12, "X22": X22, "X32": X32, "X42": X42,
                         "A": A, "Y": Y})

    return data

Generate training dataset.

In [57]:
Dtrain = generate_data()

# simulate the creation of a 'broad zero-shot predictor'
Dtrain['X1'] = (Dtrain['X11']+Dtrain['X12'])/2
Dtrain['X2'] = (Dtrain['X21']+Dtrain['X22'])/2
Dtrain['X3'] = (Dtrain['X31']+Dtrain['X32'])/2
Dtrain['X4'] = (Dtrain['X41']+Dtrain['X42'])/2

Train a linear logistic regressor to simulate a zero-shot classifier.

In [58]:
zero_shot_model = sm.GLM.from_formula(formula="U~X1+X2+X3+X4", data=Dtrain, family=sm.families.Binomial()).fit()

zero_shot_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      U   No. Observations:                20000
Model:                            GLM   Df Residuals:                    19995
Model Family:                Binomial   Df Model:                            4
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2121.1
Date:                Wed, 27 Dec 2023   Deviance:                       4242.2
Time:                        22:18:07   Pearson chi2:                 3.37e+04
No. Iterations:                     9   Pseudo R-squ. (CS):             0.6901
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -6.2729      0.122    -51.369      0.000      -6.512      -6.034
X1             2.5876      0.124     20.884      0.000       2.345       2.830
X2             0.3894      0.029     13.371      0.000       0.332       0.446
X3             1.9700      0.093     21.110      0.000       1.787       2.153
X4             0.1985      0.021      9.504      0.000       0.158       0.239
==============================================================================
"""

Generate the inference dataset, and generate Z and W by using the two realizations of the covariates.

In [59]:
DInference = generate_data()

# create the two realizations of the covariates to generate Z and W
realization1 = pd.DataFrame()
realization1['X1'] = DInference['X11']
realization1['X2'] = DInference['X21']
realization1['X3'] = DInference['X31']
realization1['X4'] = DInference['X41']

W = binary_predictions(zero_shot_model.predict(realization1))

realization2 = pd.DataFrame()
realization2['X1'] = DInference['X12']
realization2['X2'] = DInference['X22']
realization2['X3'] = DInference['X32']
realization2['X4'] = DInference['X42']

Z = binary_predictions(zero_shot_model.predict(realization2))

DInference['Z'] = Z
DInference['W'] = W

Verify odds ratio values between W and Z.

In [60]:
print("odds ratio:", odds_ratio("W", "Z", [], DInference))
print("odds ratio, condition U:", odds_ratio("W", "Z", ["U"], DInference))

odds ratio: 17.762883784542595
odds ratio, condition U: 0.8937535298321228


Proceed with the causal inference pipeline as normal.

In [61]:
print(proximal_find_ace("A", "Y", "W", "Z", [], DInference))
print(compute_confidence_intervals("A", "Y", "W", "Z", [], DInference))

1.3297527193348975
(1.2977998685669498, 1.3608747679972029)


As expected, using the same model to generate Z and W creates proxies that are valid for proximal causal inference.

## Part 2

Use the zero-shot predictor to generate a proxy W and a simple heuristic based predictor to generate the proxy Z. Then, proceed with proximal causal inference as normal.

In [62]:
DInference = generate_data()

# create the first realization of the covariates to generate W
realization1 = pd.DataFrame()
realization1['X1'] = DInference['X11']
realization1['X2'] = DInference['X21']
realization1['X3'] = DInference['X31']
realization1['X4'] = DInference['X41']

W = binary_predictions(zero_shot_model.predict(realization1))

# use a simple heuristic for the second realization of the covariates to generate the proxy Z
# (this heuristic attains roughly accuracy 0.84)
Z = []
for i in range(size):
    if DInference['X12'][i] > 1.1:
        Z.append(1)
    else:
        Z.append(0)

DInference['W'] = W
DInference['Z'] = Z

Verify odds ratio values between W and Z.

In [63]:
print("odds ratio:", odds_ratio("W", "Z", [], DInference))
print("odds ratio, condition U:", odds_ratio("W", "Z", ["U"], DInference))

odds ratio: 10.530085583788464
odds ratio, condition U: 0.9797662704009918


Proceed with proximal causal inference pipeline.

In [64]:
print(proximal_find_ace("A", "Y", "W", "Z", [], DInference))
print(compute_confidence_intervals("A", "Y", "W", "Z", [], DInference))

1.3008560647113057
(1.2694762800736616, 1.333261996955854)
